# 9-卷积神经网络（CNN）

卷积神经网络是一类良好的特征提取器，多用于图像识别领域，现在在自然语言处理领域和语音识别领域也有较为广泛的运用。

导包

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

载入数据

In [2]:
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST\t10k-images-idx3-ubyte.gz
Extracting MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


设置batch_size的大小和批次

In [3]:
batch_size = 100
n_batchs = mnist.train.num_examples // batch_size

In [4]:
n_batchs

550

设置权重初始化函数

传入shape，返回对应shape的参数，服从高斯分布，0均值，0.1方差

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

设置偏执初始化的函数

传入shape，返回对应的0.1的参数

In [6]:
def biases_vriable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

定义卷积层函数，用于生成一个对应的卷积操作

In [7]:
def  conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')

定义$2\times 2$的池化层

In [8]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

定义两个placeholder

In [9]:
x = tf.placeholder(shape=[None, 784], dtype=tf.float32)
y = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [10]:
x,y

(<tf.Tensor 'Placeholder:0' shape=(?, 784) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>)

改变x的shape

输入的为1维数据，由于使用2D卷积，所以要将数据转为2D的图像数据

In [11]:
x_images = tf.reshape(x, shape=[-1, 28, 28, 1])

In [12]:
x_images

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

搭建卷积层

In [13]:
w_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = biases_vriable([32])

h_conv1 = tf.nn.relu(conv2d(x_images, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

w_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = biases_vriable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [14]:
w_conv1, b_conv1

(<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>)

In [15]:
h_conv1,h_pool1

(<tf.Tensor 'Relu:0' shape=(?, 28, 28, 32) dtype=float32>,
 <tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>)

In [16]:
w_conv2, b_conv2

(<tf.Variable 'Variable_2:0' shape=(5, 5, 32, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>)

In [17]:
h_conv2,h_pool2

(<tf.Tensor 'Relu_1:0' shape=(?, 14, 14, 64) dtype=float32>,
 <tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>)

搭建全连接网络

同时将池化层的输出reshape为全联接的输入，并输入到网络中

In [18]:
w_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = biases_vriable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, w_fc1.shape[0]])
h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

设置Dropout

In [19]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

设置输出层

In [20]:
w_fc2 = weight_variable([1024, 10])
b_fc2 = biases_vriable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

定义损失函数与优化器

In [21]:
 loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [22]:
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

定义测评结果

In [23]:
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
init = tf.global_variables_initializer()

训练

In [25]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(100):
        for batch in range(n_batchs):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, {x: batch_xs, y:batch_ys, keep_prob:0.7})
        acc, l = sess.run([accuracy, loss], {x:mnist.test.images, y:mnist.test.labels, keep_prob: 1.0})
        print("Iter: " + str(epoch) + " Accuracy: " + str(acc) + " Loss: "+ str(l) )

Iter: 0 Accuracy: 0.0982 Loss: 2.36295
Iter: 1 Accuracy: 0.0982 Loss: 2.36295
Iter: 2 Accuracy: 0.0982 Loss: 2.36295
Iter: 3 Accuracy: 0.0982 Loss: 2.36295
Iter: 4 Accuracy: 0.0982 Loss: 2.36295
Iter: 5 Accuracy: 0.8835 Loss: 1.5764903
Iter: 6 Accuracy: 0.8892 Loss: 1.5702636
Iter: 7 Accuracy: 0.8931 Loss: 1.5673627
Iter: 8 Accuracy: 0.8921 Loss: 1.5675625
Iter: 9 Accuracy: 0.9855 Loss: 1.4760377
Iter: 10 Accuracy: 0.9876 Loss: 1.4740084
Iter: 11 Accuracy: 0.9894 Loss: 1.4724655
Iter: 12 Accuracy: 0.9899 Loss: 1.471707
Iter: 13 Accuracy: 0.988 Loss: 1.472945
Iter: 14 Accuracy: 0.9914 Loss: 1.4700143
Iter: 15 Accuracy: 0.9905 Loss: 1.4708084
Iter: 16 Accuracy: 0.9912 Loss: 1.4701136
Iter: 17 Accuracy: 0.9896 Loss: 1.4714066
Iter: 18 Accuracy: 0.9909 Loss: 1.4702921
Iter: 19 Accuracy: 0.9916 Loss: 1.4701236
Iter: 20 Accuracy: 0.9904 Loss: 1.4708687
Iter: 21 Accuracy: 0.9918 Loss: 1.469631
Iter: 22 Accuracy: 0.9893 Loss: 1.4720784
Iter: 23 Accuracy: 0.9917 Loss: 1.4694481
Iter: 24 Accurac